# Lab | Data Structuring and Combining Data

Objective: 
- Combine and integrate data from multiple sources using merging, concatenating, or joining techniques to generate more comprehensive and meaningful datasets for analysis.
- Modify the structure of data by pivoting, stacking/unstacking, or melting dataframes, enabling them to efficiently explore and analyze complex datasets. 

# Challenge 1: Combining Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the following links: 
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv


Perform data cleaning and formatting using the main cleaning and formatting function created in the previous lab to clean and format the data.

Combine the data from the three dataframes into a single dataframe, named "customer_data", using appropriate merging, concatenating, and joining techniques.

Verify that the customer_data dataframe contains all the rows and columns from the three original dataframes.

Observation: 
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [1]:
import pandas as pd
import matplotlib as mp
import numpy as np
import funciones_increible as fun

df1 = pd.read_csv('file1.csv')

df2 = pd.read_csv('file2.csv')

df3 = pd.read_csv('file3.csv')


cols = []
for i in range (len(df1.columns)):
    cols.append(df1.columns[i].lower())
df1.columns = cols

cols = []
for i in range (len(df2.columns)):
    cols.append(df2.columns[i].lower())
df2.columns = cols

cols = []
for i in range (len(df3.columns)):
    cols.append(df3.columns[i].lower())
df3.columns = cols

new_names = {'st':'state',}
df1.rename(columns=new_names,inplace=True)
df2.rename(columns=new_names,inplace=True)
df3.rename(columns=new_names,inplace=True)

full_df = pd.concat([df1,df2,df3],axis=0)
display(full_df.shape)

full_df = fun.clean_head_names(full_df)
full_df = fun.clean_invalid_values(full_df)
full_df = fun.formating_data_types(full_df)
full_df = fun.delete_null_values(full_df)
full_df = fun.delete_duplicated_values(full_df)

display(full_df.shape)

(12074, 11)

(9056, 11)

# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

## Exercise 1: Clean and Format the Data

While the dataset has been partially cleaned and formatted, we still need to perform several data cleaning tasks. Specifically, we need to standardize the column names, clean null values, convert the effective_to_date column to datetime, and extract the months from the dataset and store them in a separate column. 

To accomplish these tasks, we will use the functions created in the previous step to standardize the column names and deal with null values, and then we will apply additional functions to convert the effective_to_date column to datetime and extract the months.

Save the clean dataset into a csv file.

In [2]:
marketing_df = pd.read_csv('marketing_customer_analysis.csv')

def standardize_header(data):
    cols = []
    for i in range (len(data.columns)):
        cols.append(data.columns[i].lower())
    data.columns = cols

    data.columns = data.columns.str.replace(" ","_")

    new_names = {'unnamed:_0':'id',}
    data.rename(columns=new_names,inplace=True)

standardize_header(marketing_df)

In [3]:
marketing_df.isnull().values.any()
marketing_df.isnull().sum().sum()

9254

In [4]:
#we can delete the column because it has half of the values in null
def delete_column(data,column):
    data = data.drop([column],axis=1)
    return data

marketing_df = delete_column(marketing_df,'vehicle_type')

#fill null values of categorical columns with unknown
def fillnull_cate(data,column):
    data[column] = data[column].fillna('unknown')
    return data
marketing_df = fillnull_cate(marketing_df,'state')
marketing_df = fillnull_cate(marketing_df,'response')
marketing_df = fillnull_cate(marketing_df,'vehicle_class')
marketing_df = fillnull_cate(marketing_df,'vehicle_size')

#fill null values of numerical columns with median
def fillnull_num(data,column):
    data[column] = data[column].fillna(data[column].median())
    return data
marketing_df = fillnull_num(marketing_df,'months_since_last_claim')
marketing_df = fillnull_num(marketing_df,'number_of_open_complaints')

In [5]:
display(marketing_df.isnull().sum())

id                               0
customer                         0
state                            0
customer_lifetime_value          0
response                         0
coverage                         0
education                        0
effective_to_date                0
employmentstatus                 0
gender                           0
income                           0
location_code                    0
marital_status                   0
monthly_premium_auto             0
months_since_last_claim          0
months_since_policy_inception    0
number_of_open_complaints        0
number_of_policies               0
policy_type                      0
policy                           0
renew_offer_type                 0
sales_channel                    0
total_claim_amount               0
vehicle_class                    0
vehicle_size                     0
dtype: int64

In [6]:
marketing_df['effective_to_date']=pd.to_datetime(marketing_df['effective_to_date'], errors='coerce')
marketing_df['months']=pd.DatetimeIndex(marketing_df['effective_to_date']).month

display(marketing_df.isnull().sum())

id                               0
customer                         0
state                            0
customer_lifetime_value          0
response                         0
coverage                         0
education                        0
effective_to_date                0
employmentstatus                 0
gender                           0
income                           0
location_code                    0
marital_status                   0
monthly_premium_auto             0
months_since_last_claim          0
months_since_policy_inception    0
number_of_open_complaints        0
number_of_policies               0
policy_type                      0
policy                           0
renew_offer_type                 0
sales_channel                    0
total_claim_amount               0
vehicle_class                    0
vehicle_size                     0
months                           0
dtype: int64

## Exercise 2: Structuring Data

1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

3. You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table. 

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [32]:
# quiero saber que sales_chanel a tenido mas ventas en terminos de total_revenue


sumary_table = marketing_df.pivot(columns='sales_channel',values='total_claim_amount').round(2).max()
display(marketing_df.shape)
display(sumary_table)

(10910, 26)

sales_channel
Agent          2552.34
Branch         2345.41
Call Center    2759.79
Web            2893.24
dtype: float64

In [34]:
average_ = marketing_df.pivot_table(columns=['gender','education'],values='customer_lifetime_value').round(2).mean() 

average_

gender  education           
F       Bachelor                7874.27
        College                 7748.82
        Doctor                  7328.51
        High School or Below    8675.22
        Master                  8157.05
M       Bachelor                7703.60
        College                 8052.46
        Doctor                  7415.33
        High School or Below    8149.69
        Master                  8168.83
dtype: float64

In [43]:
quejas1 = marketing_df.groupby(['policy_type', 'months']).size().reset_index(name='number_of_complaints')

quejas2 = marketing_df.value_counts(['policy_type', 'months'])

display(quejas1)
display(quejas2)

,policy_type,months,number_of_complaints
0,Corporate Auto,1,1252
1,Corporate Auto,2,1089
2,Personal Auto,1,4329
3,Personal Auto,2,3799
4,Special Auto,1,237
5,Special Auto,2,204


,policy_type,months,number_of_complaints
0,Personal Auto,1,4329
1,Personal Auto,2,3799
2,Corporate Auto,1,1252
3,Corporate Auto,2,1089
4,Special Auto,1,237
5,Special Auto,2,204
